# Imports

In [ ]:
%pip install panda-gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925.5/925.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.5/80.5 MB 18.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pybullet: filename=pybullet-3.2.5-cp310-cp310-linux_x86_64.whl size=101451201 sha256=82a8a778fbc1a7884120ad0680bcec345e69c4739a142139effd21b42669b5e5
  Stored in directory: /root/.cache/pip/wheels/6b/fa/1a/c315a5133f0c9bf202a6daa5d70891120e7fe403e06e3407cc
Successfully built pybullet


In [ ]:
import sys
sys.path.append("../")

import pickle

import matplotlib.pyplot as plt
from matplotlib import collections as mc
import numpy as np
# import sklearn
# from sklearn import linear_model
import torch

# from latent_actions.cvae.vae import VAE
# from latent_actions.envs.panda_center_out import PandaCenterOutEnv
import pandas as pd
torch.manual_seed(0)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import TensorDataset,DataLoader

# import torchvision.transforms as transforms
# import torchvision.datasets as datasets

from sklearn import metrics
from sklearn import decomposition
from sklearn import manifold
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import TensorDataset,DataLoader
import copy
import random
import time

import argparse
import numpy as np

import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image
import matplotlib.pyplot as plt
from PIL import Image

import numpy as np
from panda_gym.envs.tasks.reach import Reach

In [ ]:
import numpy as np
from panda_gym.envs.tasks.reach import Reach


class CenterOut(Reach):
    """The center out task is a specific reaching task where the goal is
    confined to the level surface of the end effector."""

    def __init__(
        self,
        sim,
        get_ee_position,
        reward_type = "sparse",
        distance_threshold = 0.05,
        goal_range = 0.3,
        dimension = 2,
    ) -> None:
        super().__init__(
                sim, get_ee_position, reward_type, distance_threshold,
                goal_range)
        assert(dimension == 2 or dimension == 3)
        self.dimension = dimension

    def set_ee_position(self, ee_position: np.ndarray) -> None:
        """End effector position is the center of the grid."""
        self.ee_position = ee_position

    def _sample_goal(self) -> np.ndarray:
        goal_displacement_from_center = np.zeros(self.dimension)
        while all(goal_displacement_from_center == 0):
            goal_displacement_from_center = self.np_random.choice(
                    [self.goal_range_low[0], self.goal_range_high[0], 0],
                    self.dimension)

        if self.dimension == 2:
            goal_displacement_from_center = np.append(
                    goal_displacement_from_center, 0)
        goal = self.ee_position + goal_displacement_from_center
        return goal



In [ ]:
import numpy as np

from panda_gym.envs.core import RobotTaskEnv
from panda_gym.envs.robots.panda import Panda
from panda_gym.pybullet import PyBullet

# from latent_actions.envs.center_out import CenterOut

class PandaCenterOutEnv(RobotTaskEnv):
    """Reach task wih Panda robot.
    Args:
        render (bool, optional): Activate rendering. Defaults to False.
        reward_type (str, optional): "sparse" or "dense". Defaults to "sparse".
        control_type (str, optional): "ee" to control end-effector position or "joints" to control joint values.
            Defaults to "ee".
    """

    def __init__(self,
            render: bool = False,
            reward_type: str = "sparse",
            control_type: str = "ee",
            goal_range: int = 0.3,
            base: int = [0.0,0.0,0.0],
            dimension: int = 2) -> None:
        sim = PyBullet()
        robot = Panda(
                sim, block_gripper=True,
                base_position=np.array(base),
                control_type=control_type)
        robot.reset() # Brings the robot to neutral starting position.
        task = CenterOut(
                sim, reward_type=reward_type,
                get_ee_position=robot.get_ee_position, goal_range=goal_range,
                dimension=dimension)
        task.set_ee_position(robot.get_ee_position())
        super().__init__(robot, task)



# Data

## Simple Cyclic

start:tensor([ 0.3350, -0.0200, -0.0700])

data categories: joint_angles, actions_joints, ee_positions, ee_velocities

usage:
train_loader = DataLoader(TensorDataset(ee_velocities, ee_positions), batch_size=128)

test_loader = DataLoader(TensorDataset(ee_velocities, ee_positions), batch_size=128)

train_data_size = np.array(next(iter(train_loader)))[0].shape

train_label_size = np.array(next(iter(train_loader)))[1].shape

**function**:

train, test = simplecyclicdata(inp='joints', ctxt='ee', batch_size=128)



In [ ]:
import copy
def add_noise(traj, sigma, ratio):
    new_traj = copy.deepcopy(traj)
    steps_affected = np.random.randint(0, len(new_traj), size=int(len(new_traj)*ratio))
    for step in steps_affected:
        noise = np.random.normal(0, sigma, 3)
        new_traj[step] += noise

    return new_traj

In [1]:
def convertdata(dataname: str, actiontype: str, ctxttype: str, noise: bool):
  with open(dataname+'_train_data.npy', 'rb') as fp:
      train_episodes = np.load(fp)
  with open(dataname+'_train_label.npy', 'rb') as fp:
      train_labels = np.load(fp)
  with open(dataname+'_test_data.npy', 'rb') as fp:
      test_episodes = np.load(fp)
  with open(dataname+'_test_label.npy', 'rb') as fp:
      test_labels = np.load(fp)
  env = PandaCenterOutEnv(render=False, control_type='ee',base=[0.0,0.0,0.0])

  if noise:                     # ee pos                 # gripper v
    # clean this up by creating a function to add noise
    mega_test = np.hstack((test_episodes[:,4:], test_labels[:,-1].reshape(-1,1)))
    for i in np.arange(10):
      test = add_noise(test_episodes[:,4:7], 1e-2, 0.3) # ee positions (3)
      test = np.hstack((test, test_episodes[:,-1].reshape(-1,1),  test_labels[:,-1].reshape(-1,1))) # stack with grip v and grip width; results bad if add noise to grip
      mega_test = np.vstack((mega_test, test)) # unnoised + noised
    test_labels = mega_test.copy()

    mega_train = np.hstack( (train_episodes[:,4:],train_labels[:,-1].reshape(-1,1)) )
    for i in np.arange(10):
      train = add_noise(train_episodes[:,4:7],1e-2,0.3)
      train = np.hstack( (train, train_episodes[:,-1].reshape(-1,1),  train_labels[:,-1].reshape(-1,1)) )
      mega_train = np.vstack( (mega_train, train) )
    train_labels = mega_train.copy()

    # inverse kinematics stuff to obtain velocities from positions
    joint_angles_test = np.array( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0,0.0])) for ee_pos in mega_test[:,:-2]])[:,:7] # -2 to ignore gripper w & v (dont need IK for gripper); first 7 joints from panda gym
    actions_joints_test = np.vstack( ( (joint_angles_test[1:] - joint_angles_test[:-1]), np.zeros((1,7)) )) # x1 - x0 / t1 - t0 = velocity
    joint_angles_train = np.array( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0,0.0])) for ee_pos in mega_train[:,:-2]])[:,:7]
    actions_joints_train = np.vstack( ( (joint_angles_train[1:] - joint_angles_train[:-1]), np.zeros((1,7)) ))

    if actiontype == 'joints':
      train_labels = np.hstack( (actions_joints_train, mega_train[:,-1].reshape(-1,1)))
      test_labels = np.hstack( (actions_joints_test, mega_test[:,-1].reshape(-1,1)))
      print(train_labels.shape, test_labels.shape)
      np.save(dataname+'_train_label_'+'joints'+'.npy', train_labels)
      np.save(dataname+'_test_label_'+'joints'+'.npy', test_labels)

    if ctxttype == 'joints':
      train_episodes = np.hstack( (train_episodes[:,:4], joint_angles_train, train_episodes[:,-1].reshape(-1,1)))
      test_episodes = np.hstack( (test_episodes[:,:4], joint_angles_test, test_episodes[:,-1].reshape(-1,1)))
      np.save(dataname+'_train_episode_'+'joints'+'.npy', train_episodes)
      np.save(dataname+'_test_episode_'+'joints'+'.npy', test_episodes)

  else:
    joint_angles_test = np.array( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0,0.0])) for ee_pos in test_episodes[:,4:7]])[:,:7]
    actions_joints_test = np.vstack( ( (joint_angles_test[1:] - joint_angles_test[:-1]), np.zeros((1,7)) ))
    joint_angles_train = np.array( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0,0.0])) for ee_pos in train_episodes[:,4:7]])[:,:7]
    actions_joints_train = np.vstack( ( (joint_angles_train[1:] - joint_angles_train[:-1]), np.zeros((1,7)) ))

    if actiontype == 'joints':
      train_labels = np.hstack( (actions_joints_train, train_labels[:,-1].reshape(-1,1)))
      test_labels = np.hstack( (actions_joints_test, test_labels[:,-1].reshape(-1,1)))
      np.save(dataname+'_train_label_'+'joints'+'.npy', train_labels)
      np.save(dataname+'_test_label_'+'joints'+'.npy', test_labels)

    if ctxttype == 'joints':
      train_episodes = np.hstack( (train_episodes[:,:4], joint_angles_train, train_episodes[:,-1].reshape(-1,1)))
      test_episodes = np.hstack( (test_episodes[:,:4], joint_angles_test, test_episodes[:,-1].reshape(-1,1)))
      np.save(dataname+'_train_episode_'+'joints'+'.npy', train_episodes)
      np.save(dataname+'_test_episode_'+'joints'+'.npy', test_episodes)


In [ ]:
# convertdata('cyclic_traj_simple', 'joints', 'ee')
# convertdata('cyclic_traj_with_grasping', 'joints', 'ee', True)
# convertdata('cyclic_traj_with_grasping_noise', 'joints', 'ee')
# convertdata('cyclic_traj_multiple_starts', 'joints', 'ee', True) # joint vels conditioned on ee pos context

In [ ]:
t = np.zeros((2,7))
print(t)
print(t.reshape(1,-1).shape)

In [ ]:
plt.plot(joint_angles[:,0][:10000])
joint_angles[:,0].shape

In [ ]:
plt.plot(joint_angles[:,3][:10000])
joint_angles[:,3].shape

In [ ]:
# # dataset = TensorDataset(joint_velocities_train)
# # dataset = joint_velocities
# train_loader = DataLoader(TensorDataset(ee_velocities, ee_positions), batch_size=128)
# test_loader = DataLoader(TensorDataset(ee_velocities, ee_positions), batch_size=128)
# train_data_size = np.array(next(iter(train_loader)))[0].shape
# train_label_size = np.array(next(iter(train_loader)))[1].shape

In [ ]:
def simplecyclicdata(inp='joints', ctxt='ee', batch_size=128):
  print('simple cyclic')

  with open('cyclic_traj_simple_train_data.npy', 'rb') as fp:
      train_episodes = np.load(fp)
  with open('cyclic_traj_simple_train_label.npy', 'rb') as fp:
      train_labels = np.load(fp)
  with open('cyclic_traj_simple_test_data.npy', 'rb') as fp:
      test_episodes = np.load(fp)
  with open('cyclic_traj_simple_test_label.npy', 'rb') as fp:
      test_labels = np.load(fp)

  env = PandaCenterOutEnv(render=False, control_type='ee')

  joint_angles = torch.Tensor( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0])) for ee_pos in train_episodes[:,4:7]])[:,:7]
  actions_joints = torch.vstack( ( (joint_angles[1:] - joint_angles[:-1]), torch.zeros((1,7)) ))
  ee_positions = torch.Tensor(train_episodes[:,4:7])
  ee_velocities = torch.Tensor(train_labels[:,:-1])
  print(ee_positions.shape, ee_velocities.shape, joint_angles.shape, actions_joints.shape)
  print(ee_positions[0])

  if inp=='ee':
    X = ee_velocities
  elif inp=='joints':
    X = actions_joints

  if ctxt=='ee':
    c = ee_positions
  elif ctxt=='joints':
    c = joint_angles
  X = torch.Tensor(X)
  y = X

  if ctxt=='none':
    print('no context')
    traind = DataLoader(TensorDataset(X,y), batch_size=batch_size)

  else:
    c = torch.Tensor(c)
    # X = torch.hstack( (X, c))
    traind = DataLoader(TensorDataset(X,c), batch_size=batch_size)

  testd = traind #fix later
  return traind, testd

# #uncomment to test
# train, test = simplecyclicdata(inp = 'ee', ctxt='none')

## cyclic grasp

start: tensor([ 0.3350, -0.0200, -0.0700])

data categories: joint_angles, actions_joints, ee_positions, ee_velocities

usage:
train_loader = DataLoader(TensorDataset(ee_velocities, ee_positions), batch_size=128)

test_loader = DataLoader(TensorDataset(ee_velocities, ee_positions), batch_size=128)

train_data_size = np.array(next(iter(train_loader)))[0].shape

train_label_size = np.array(next(iter(train_loader)))[1].shape

train, test = graspcyclicdata(inp='joints', ctxt='ee', batch_size=128)

In [ ]:

with open('cyclic_traj_with_grasping_train_data.npy', 'rb') as fp:
    train_episodes = np.load(fp)
with open('cyclic_traj_with_grasping_train_label.npy', 'rb') as fp:
    train_labels = np.load(fp)
with open('cyclic_traj_with_grasping_test_data.npy', 'rb') as fp:
    test_episodes = np.load(fp)
with open('cyclic_traj_with_grasping_test_label.npy', 'rb') as fp:
    test_labels = np.load(fp)

env = PandaCenterOutEnv(render=False, control_type='ee')

joint_angles_train = torch.Tensor( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0])) for ee_pos in train_episodes[:,4:7]])[:,:7]
joint_velocities_train = torch.vstack( ( (joint_angles_train[1:] - joint_angles_train[:-1]), torch.zeros((1,7)) ))
ee_positions_train = torch.Tensor(train_episodes[:,4:7  ])
ee_velocities_train = torch.Tensor(train_labels[:,:-1])
gripper_positions_train = torch.Tensor( train_episodes[:-1])
gripper_velocities_train = torch.Tensor( train_labels[:-1])
print(ee_positions_train.shape, ee_velocities_train.shape, joint_angles_train.shape, joint_velocities_train.shape, gripper_positions_train.shape, gripper_velocities_train.shape)




joint_angles_test = torch.Tensor( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0])) for ee_pos in test_episodes[:,4:7]])[:,:7]
joint_velocities_test = torch.vstack( ( (joint_angles_test[1:] - joint_angles_test[:-1]), torch.zeros((1,7)) ))
ee_positions_test = torch.Tensor(test_episodes[:,4:7])
ee_velocities_test = torch.Tensor(test_labels[:,:-1])
gripper_positions_test = torch.Tensor( test_episodes[:-1])
gripper_velocities_test = torch.Tensor( test_labels[:-1])
print(ee_positions_test.shape, ee_velocities_test.shape, joint_angles_test.shape, joint_velocities_test.shape, gripper_positions_test.shape, gripper_velocities_test.shape)
print(ee_positions_train[0], ee_positions_test[0])


In [ ]:
# dataset = TensorDataset(joint_velocities_train)
# dataset = joint_velocities
train_loader = DataLoader(TensorDataset(ee_velocities_train, ee_positions_train), batch_size=128)
test_loader = DataLoader(TensorDataset(ee_velocities_test, ee_positions_test), batch_size=128)
train_data_size = np.array(next(iter(train_loader)))[0].shape
train_label_size = np.array(next(iter(train_loader)))[1].shape

In [ ]:
def graspcyclicdata(inp='joints', ctxt='ee', batch_size=128):


  with open('cyclic_traj_with_grasping_train_data.npy', 'rb') as fp:
      train_episodes = np.load(fp)
  with open('cyclic_traj_with_grasping_train_label.npy', 'rb') as fp:
      train_labels = np.load(fp)
  with open('cyclic_traj_with_grasping_test_data.npy', 'rb') as fp:
      test_episodes = np.load(fp)
  with open('cyclic_traj_with_grasping_test_label.npy', 'rb') as fp:
      test_labels = np.load(fp)

  env = PandaCenterOutEnv(render=False, control_type='ee')

  joint_angles_train = torch.Tensor( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0])) for ee_pos in train_episodes[:,4:7]])[:,:7]
  joint_velocities_train = torch.vstack( ( (joint_angles_train[1:] - joint_angles_train[:-1]), torch.zeros((1,7)) ))
  ee_positions_train = torch.Tensor(train_episodes[:,4:7  ])
  ee_velocities_train = torch.Tensor(train_labels[:,:-1])
  gripper_positions_train = torch.Tensor( train_episodes[:-1])
  gripper_velocities_train = torch.Tensor( train_labels[:-1])
  print(ee_positions_train.shape, ee_velocities_train.shape, joint_angles_train.shape, joint_velocities_train.shape, gripper_positions_train.shape, gripper_velocities_train.shape)




  joint_angles_test = torch.Tensor( [env.robot.inverse_kinematics(link=11, position=ee_pos, orientation=np.array([1.0,0.0,0.0])) for ee_pos in test_episodes[:,4:7]])[:,:7]
  joint_velocities_test = torch.vstack( ( (joint_angles_test[1:] - joint_angles_test[:-1]), torch.zeros((1,7)) ))
  ee_positions_test = torch.Tensor(test_episodes[:,4:7])
  ee_velocities_test = torch.Tensor(test_labels[:,:-1])
  gripper_positions_test = torch.Tensor( test_episodes[:-1])
  gripper_velocities_test = torch.Tensor( test_labels[:-1])
  print(ee_positions_test.shape, ee_velocities_test.shape, joint_angles_test.shape, joint_velocities_test.shape, gripper_positions_test.shape, gripper_velocities_test.shape)
  print(ee_positions_train[0], ee_positions_test[0])


  print('grasp cyclic')
  if inp=='ee':
    Xtrain = ee_velocities_train
    Xtest = ee_velocities_test
  elif inp=='joints':
    Xtrain = joint_velocities_train
    Xtest = joint_velocities_test

  Xtrain = torch.Tensor(Xtrain)
  Xtest = torch.Tensor(Xtest)

  ytrain= Xtrain
  ytest = Xtest

  if ctxt=='none':
    print('no context')
    traind = DataLoader(TensorDataset(Xtrain,ytrain), batch_size=batch_size)
    testd = DataLoader(TensorDataset(Xtrain,ytrain), batch_size=batch_size)

  else:
    if ctxt=='ee':
      ctrain = ee_positions_train
      ctest = ee_positions_test

    elif ctxt=='joints':
      ctrain = joint_angles_train
      ctest = joint_angles_test

    ctrain = torch.Tensor(ctrain)
    ctest = torch.Tensor(ctest)

    traind = DataLoader(TensorDataset(Xtrain,ctrain), batch_size=batch_size)
    testd = DataLoader(TensorDataset(Xtest,ctest), batch_size=batch_size)

  return traind, testd

# #uncomment to test
# train, test = graspcyclicdata(inp = 'ee', ctxt='ee')
# print(next(iter(train))[0].shape)

# device

device call: device

In [ ]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"
device = torch.device(dev)

#M O D E L S

### random fourier feature mapping:
https://bmild.github.io/fourfeat/

ffmap(inputdata, outputdim, std)

output: [ cos(.*data) , sin(.*data) ]

In [ ]:
import math
def ffmap(data, outdim, sigma=1):
    mean = torch.zeros((data.shape[1],outdim))
    std = torch.ones_like(mean) * sigma
    g = torch.normal(mean, std)
    # print(g.shape)
    out = torch.matmul(data, g)
    out = torch.hstack( (torch.cos(2*math.pi*out), torch.sin(2*math.pi*out)) )
    return out

## VAE_gumbel(temp=0.5, action_dim=3, context_dim=3, dims=50)

In [ ]:
if torch.cuda.is_available():
  dev = "cuda:0"
else:
  dev = "cpu"

is_cuda = torch.device(dev)
torch.manual_seed(0)

if is_cuda:
    torch.cuda.manual_seed(0)


latent_dim = 1
categorical_dim = 4  # one-of-K vector

hard = False

# log_interval = 10
log_interval = 100
ANNEAL_RATE = 0.00003
temp_min = 0.5

In [ ]:
def sample_gumbel(shape, eps=1e-7):
    U = torch.rand(shape)
    if is_cuda:
        U = U.cuda()
    return -torch.log(-torch.log(U + eps) + eps)


def gumbel_softmax_sample(logits, temperature):
    y = logits + sample_gumbel(logits.size())
    return F.softmax(y / temperature, dim=-1)

# latent_dim = 1
# categorical_dim = 4
def gumbel_softmax(logits, temperature, hard=False):
    """
    ST-gumple-softmax
    input: [*, n_class]
    return: flatten --> [*, n_class] an one-hot vector
    """
    y = gumbel_softmax_sample(logits, temperature)

    if not hard:
        return y.view(-1, latent_dim * categorical_dim)

    shape = y.size()
    _, ind = y.max(dim=-1)
    y_hard = torch.zeros_like(y).view(-1, shape[-1])
    y_hard.scatter_(1, ind.view(-1, 1), 1)
    y_hard = y_hard.view(*shape)
    # Set gradients w.r.t. y_hard gradients w.r.t. y
    y_hard = (y_hard - y).detach() + y
    return y_hard.view(-1, latent_dim * categorical_dim)

class VAE_gumbel(nn.Module):
    def __init__(self, temp=0.5, action_dim=3, context_dim=3, dims=50):
        super(VAE_gumbel, self).__init__()
        self.fc1 = nn.Linear(action_dim+context_dim, dims)
        self.bn1 = nn.BatchNorm1d(dims)
        self.fc2 = nn.Linear(dims, dims)
        self.bn2 = nn.BatchNorm1d(dims)
        self.fc3 = nn.Linear(dims, latent_dim * categorical_dim)

        self.fc4 = nn.Linear(context_dim+(latent_dim * categorical_dim), dims)
        self.bn4 = nn.BatchNorm1d(dims)
        self.fc5 = nn.Linear(dims, dims)
        self.bn5 = nn.BatchNorm1d(dims)

        self.fc6 = nn.Linear(dims, action_dim)

        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.bn1(self.fc1(x)))
        h2 = self.relu(self.bn2(self.fc2(h1)))
        return self.relu(self.fc3(h2))

    def decode(self, z, label):
        # print(z.shape, label.shape)
        h4 = self.relu(self.bn4(self.fc4(torch.hstack( (z,label) ))))
        h5 = self.relu(self.bn5(self.fc5(h4)))
        return (self.fc6(h5))

    def forward(self, x, temp, hard, label):
        x = torch.hstack((x,label))
        q = self.encode(x.view(-1, x.shape[-1]))
        q_y = q.view(q.size(0), latent_dim, categorical_dim)
        z = gumbel_softmax(q_y, temp, hard)
        return self.decode(z,label), F.softmax(q_y, dim=-1).reshape(*q.size())

# VAE_gumbel()

# training loop and evals

### cyclic simple/grasp/etc

end [[0.537, -0.02, -0.07],[0.335, -0.19, -0.07]]

start [ 3.8439669e-02 -2.1944723e-12  1.9740014e-01] (using env.step)

init = np.array([0.335, -0.02, -0.07]) (cyclic trajs)
goals = np.array([[0.335, -0.19, -0.07], [0.537, -0.02, -0.07]])

dirs = torch.Tensor([[1,0,0,0],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
dir_names = ['Up', 'Right', 'Down', 'Left']

neutral_joint_values = np.array([0.000000, 0.410000, 0.0000000000, -1.850000000, 0.00000000, 2.26000000, 0.7900000000, 0.00000000, 0.00000000])

 run(lr=5e-3, ANNEAL_RATE=ANNEAL_RATE, temp_min=temp_min, epochs=100):

In [ ]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, qy, action_dim=3, alpha = 0.001):
    # BCE = F.binary_cross_entropy(recon_x, x.vi  ew(-1, 784), size_average=False) / x.shape[0]
    # BCE = F.binary_cross_entropy(recon_x, x.view(-1, 10), size_average=False) / x.shape[0]

    log_ratio = torch.log(qy * categorical_dim + 1e-7)
    KLD = torch.sum(qy * log_ratio, dim=-1).mean() #mean matters here
    # print(KLD.shape)
    # print( torch.sum(qy * log_ratio, dim=-1),  torch.sum(qy * log_ratio, dim=-1).mean() )
    recon = F.mse_loss(recon_x, x.view(-1, action_dim)).mean() #mean here is meaningless
    # print(F.mse_loss(recon_x, x.view(-1, action_dim)), F.mse_loss(recon_x, x.view(-1, action_dim)).mean())
    h1 = qy * torch.log(qy + 1e-7)
    h2 = qy * np.log(1. / categorical_dim + 1e-7)
    kl = torch.mean(torch.sum(h1-h2, dim=-1), dim=0)
    alpha = 0.001
    beta = 1
    return (beta * recon) + (alpha * kl)

def train(model, epoch, m, lr=5e-3, ANNEAL_RATE=3e-5, temp_min=0.5, kl=0.001):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.cuda()
    model.train()
    train_loss = 0
    temp = temperature
    # for batch, (X,y) in enumerate(train_loader):
      # print(batch, X.shape, y.shape)
      # break
    for batch_idx, (data, label) in enumerate(train_loader):
        if is_cuda:
            data = data.cuda()
            label = label.cuda()
        optimizer.zero_grad()
        recon_batch, qy = model(data, temp, hard, label)
        loss = loss_function(recon_batch, data, qy, data.shape[-1], kl)
        # print(loss.shape, len(data), len(train_loader.dataset))
        loss.backward()
        train_loss += loss.item() * len(data)
        optimizer.step()
        if batch_idx % 100 == 1:
            temp = np.maximum(temp * np.exp(-ANNEAL_RATE * batch_idx), temp_min)

        # if batch_idx % log_interval == 0:
        #     print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #         epoch, batch_idx * len(data), len(train_loader.dataset),
        #                100. * batch_idx / len(train_loader),
        #                loss.item()))
    if epoch%100 == 0:
        print('====> Epoch: {} Average loss: {:.4f}'.format(
            epoch, train_loss / len(train_loader.dataset)))

def test(model, epoch, temp_min=0.5):
    model.eval()
    test_loss = 0
    temp = temperature
    # for i, (data, label) in enumerate(test_loader):
    #     print(data.shape, label.shape, i)
    for i, (data, label) in enumerate(test_loader):
        if is_cuda:
            data = data.cuda()
            label = label.cuda()
        recon_batch, qy = model(data, temp, hard, label)
        test_loss += loss_function(recon_batch, data, qy).item() * len(data)
        if i % 100 == 1:
            temp = np.maximum(temp * np.exp(-ANNEAL_RATE * i), temp_min)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

# temperature = 1.0
temperature = 0.5
temp_min = 0.5
ANNEAL_RATE = 0.00003

# fdsa = VAE_gumbel(temp=temperature, action_dim=train_data_size[-1], context_dim=train_label_size[-1], dims = 50)
# if is_cuda:
    # fdsa.cuda()
# train = 1
# test = 1
def run(model, lr=5e-3, ANNEAL_RATE=ANNEAL_RATE, temp_min=temp_min, epochs=100, kl = 0.001):
    for epoch in range(1, epochs + 1):
        train(model, epoch, lr, ANNEAL_RATE, temp_min, kl)
        # test(model, epoch)
# train_loader, test_loader = graspcyclicdata(inp = 'ee', ctxt='ee', batch_size=64)

# #uncomment to test
# run(fdsa, epochs=100)